In [0]:
%%bash
wget -O "mazes.zip" "https://drive.google.com/uc?export=download&id=1mfO9IOHo-KEZApbQb7svy8OYerYI24vH" >/dev/null 2>&1
unzip "mazes.zip" >/dev/null 2>&1

In [0]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from random import randrange
from time import sleep

In [0]:
ANIMATION = True
ANIMATION_FPS = 5
FILENAME = "36.txt"

START = "S"
END = "E"
WALL = "X"
PATH = "o"
OPEN_NODE = "#"
VISITED_NODE = "*"

In [0]:
class Node:
  def __init__(self, parent, position, end):
    # keeping track of parent for the purposes of creating a path
    self.parent = parent;
    self.x, self.y = position
    # compute heuristic for greedy search & a star
    self.heuristic = abs(self.x - end[0]) + abs(self.y - end[1])
    # compute distance for dijkstra & a star
    self.dist = parent.dist + 1 if parent is not None else 0

In [0]:
class BaseAlgorithm:
  def __init__(self):
    # variables for visualization
    self.output = widgets.Output()
    step_button = widgets.Button(description="Step")
    run_button = widgets.Button(description="Run")
    step_button.on_click(self.step)
    run_button.on_click(self.run)
    display(widgets.HBox((step_button, run_button)), self.output)
    self.board = []

    # variables for the algorithm
    self.start = []
    self.end = []
    self.open_nodes = []
    self.visited_nodes = []
    self.graph = []
    self.path = []
    self.finished = False


  def read_board(self, filename):
    board_heigth = sum(1 for line in open(filename)) - 2
    with open(filename) as f:
      # reads the board
      for _ in range(board_heigth):
        self.board.append(f.readline())
      self.board = list(map(list, self.board))

      # reads start and end coordinates
      self.start = [int(pos) for pos in f.readline()[6:].split(",")]
      self.end = [int(pos) for pos in f.readline()[4:].split(",")]
    
    # creates an empty graph and marks the walls
    self.graph = [[[] for _ in self.board[0]] for _ in self.board]
    for y, row in enumerate(self.board):
      for x, char in enumerate(row):
        if char == 'X':
          self.graph[y][x] = [None]

    # processes the first node and shows the board
    self.new_node(self.start[0], self.start[1], None)
    self.update_board()


  # step function, gets called by the button
  # selects node, expands it, saves it and visualizes the step
  def step(self, _=None):
    if not self.finished:
      n = self.select_node()
      self.expand_node(n)
      self.visited_nodes.append(n)
      self.update_board()


  # does steps until the end node is found
  def run(self, _=None):
    while not self.finished:
      self.step()
      sleep(ANIMATION / ANIMATION_FPS)


  def expand_node(self, node):
    # goes throught the neighboring nodes
    for x, y in zip([-1,0,1,0],[0,1,0,-1]):
      y += node.y
      x += node.x

      # if the neighboring node hasn't been found yet
      if len(self.graph[y][x]) == 0:
        self.new_node(x, y, node)
        
      else:
        neighbor_node = self.graph[y][x][0]
        # the neighboring node is not a wall and it has been found but not expanded yet
        if neighbor_node is not None and neighbor_node in self.open_nodes:
          # check if the distance to this node is now shorter
          # (this is for dijkstra but it will never get executed because
          #  all the edges in these mazes have the length of 1)
          if node.dist + 1 < neighbor_node.dist:
            # update the distance and the priority queue
            neighbor_node.dist = node.dist + 1
            self.open_nodes.sort(key=lambda n: n.dist)


  # initializes a newly expanded node
  def new_node(self, x, y, parent):
        # initializes the node and adds it to the graph & open_nodes
        n = Node(parent, [x, y], self.end)
        self.graph[y][x].append(n)
        self.open_nodes.append(n)

        # creates a path and ends, if it is the end node
        if n.x == self.end[0] and n.y == self.end[1]:
          self.finished = True
          self.create_path(n)


  # creates a path of nodes from the end node to the start node
  def create_path(self, end_node):
    p = end_node.parent
    while p is not None and p.parent is not None:
      self.path.append(p)
      p = p.parent


  def update_board(self):
      if ANIMATION or self.finished:
        with self.output:
            clear_output()

            # marks all visited nodes
            for n in self.visited_nodes:
              self.board[n.y][n.x] = VISITED_NODE

            # marks all open nodes
            for n in self.open_nodes:
              self.board[n.y][n.x] = OPEN_NODE
            
            # marks the path (if there is one)
            for n in self.path:
              self.board[n.y][n.x] = PATH
            
            # marks the start and the end
            self.board[self.start[1]][self.start[0]] = START
            self.board[self.end[1]][self.end[0]] = END

            # prints the board
            for line in self.board:
              print("".join(line[:len(line) - 1]))
            
            # prints status
            print("---------------------------------------")
            print("Nodes expanded: " + str(len(self.visited_nodes)))
            print("Path length: " + str(len(self.path)))
            


In [0]:
class RandomSearch(BaseAlgorithm):
  
  # selects a random open node
  def select_node(self):
    return self.open_nodes.pop(randrange(0, len(self.open_nodes)))

RandomSearch().read_board(FILENAME)

Output()

In [0]:
class GreedySearch(BaseAlgorithm):
  
  # selects the open node with the best heuristic score (lower is better)
  def select_node(self):
    best_i = 0;
    for i in range(len(self.open_nodes)):
      if self.open_nodes[i].heuristic < self.open_nodes[best_i].heuristic:
        best_i = i
    return self.open_nodes.pop(best_i)

GreedySearch().read_board(FILENAME)

Output()

In [0]:
class DFS(BaseAlgorithm):
  
  # a simple stack
  def select_node(self):
    return self.open_nodes.pop()

DFS().read_board(FILENAME)

Output()

In [0]:
class BFS(BaseAlgorithm):
  
  # a simple queue
  def select_node(self):
    return self.open_nodes.pop(0)

BFS().read_board(FILENAME)

Output()

In [0]:
class Dijkstra(BaseAlgorithm):
  
  # selects the open node with the lowest current distance
  def select_node(self):
    return self.open_nodes.pop(0)

Dijkstra().read_board(FILENAME)

Output()

In [0]:
class AStar(BaseAlgorithm):
  
  # selects the open node with the lowest sum
  # of the current distance + the heuristic
  def select_node(self):
    best_i = 0
    best_n = self.open_nodes[best_i]
    for i in range(len(self.open_nodes)):
      n = self.open_nodes[i]
      if n.dist + n.heuristic < best_n.dist + best_n.heuristic:
        best_i = i
        best_n = n
    return self.open_nodes.pop(best_i)

AStar().read_board(FILENAME)

Output()